# Laboratorium: Analiza obrazów przy pomocy sieci konwolucyjnych

## Ładowanie danych

Do załadowania danych skorzystamy z pakietu Tensorflow Datasets, który udostępnia wiele zbiorów przydatnych do uczenia maszynowego. Aby utrzymać względnie krótkie czasy uczenia, do ćwiczeń będziemy używać zbioru tf_flowers:

In [2]:
import tensorflow_datasets as tfds

[test_set_raw, valid_set_raw, train_set_raw], info = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True,
    with_info=True)


Kilka słów o argumentach metody load:
- split zapewnia odpowiedni podział zbioru (dlatego pierwszy element zwracanej krotki jest 3-elementowym słownikiem),
- as_supervised sprawia, że zwracane obiekty tf.data.Dataset mają postać krotek zawierających zarówno cechy, jak i etykiety,
- with_info dodaje drugi element zwracanej krotki.


In [3]:
info

tfds.core.DatasetInfo(
    name='tf_flowers',
    full_name='tf_flowers/3.0.1',
    description="""
    A large set of images of flowers
    """,
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    data_path='~\\tensorflow_datasets\\tf_flowers\\3.0.1',
    file_format=tfrecord,
    download_size=218.21 MiB,
    dataset_size=221.83 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=5),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=3670, num_shards=2>,
    },
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
)

Możemy łatwo wyekstrahować istotne parametry zbioru:

In [4]:
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes
dataset_size = info.splits["train"].num_examples

Wyświetlmy kilka przykładowych obrazów:

In [5]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
index = 0
sample_images = train_set_raw.take(9)
for image, label in sample_images:
    index += 1
    plt.subplot(3, 3, index)
    plt.imshow(image)
    plt.title("Class: {}".format(class_names[label]))
    plt.axis("off")
plt.show(block=False)

## Budujemy prostą sieć CNN

W tym ćwiczeniu zbudujemy sieć o nieskompikowanej strukturze.

###  Przygotowanie danych

Sieć będzie przetwarzała obrazy o rozmiarze 224 × 224 pikseli, a więc pierwszym krokiem będzie
przetworzenie. Obiekty Dataset pozwalają na wykorzystanie metody map, która przy uczeniu
nadzorowanym będzie otrzymywała dwa argumenty (cechy, etykieta) i powinna zwracać je w postaci
krotki po przetworzeniu.

Najprostsza funkcja będzie po prostu skalowała obraz do pożądanego rozmiaru:

In [6]:
import tensorflow as tf


def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    return resized_image, label


Aplikujemy ją do pobranych zbiorów:

In [7]:
batch_size = 12
train_set = train_set_raw.map(preprocess).shuffle(dataset_size).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

Wykorzystujemy tu dodatkowe metody Dataset API tak aby dostarczanie danych nie stało się wąskim gardłem procesu uczenia:
- shuffle losowo ustawia kolejność próbek w zbiorze uczącym,
- batch łączy próbki we wsady o podanej długości (idealnie, powinna to być wielkość miniwsadu podczas uczenia),
- prefetch zapewnia takie zarządzanie buforem, aby zawsze przygotowane było 𝑛 próbek gotowych do pobrania (w tym przypadku chcemy, aby podczas przetwarzania miniwsadu przez algorytm uczenia zawsze czekał jeden przygotowany kolejny miniwsad).


Wyświetlmy próbkę danych po przetworzeniu:

In [8]:
# plt.figure(figsize=(8, 8))
# sample_batch = train_set.take(1)
# for X_batch, y_batch in sample_batch:
#     for index in range(12):
#         plt.subplot(3, 4, index + 1)
#         plt.imshow(X_batch[index] / 255.0)
#         plt.title("Class: {}".format(class_names[y_batch[index]]))
#         plt.axis("off")
# plt.show()

### Budowa sieci

Zaprojektuj prostą sieć konwolucyjną, która pozwoli na uzyskanie przyzwoitej dokładności klasy- fikacji przetwarzanego zbioru.

Pamiętaj o istotnych zasadach:
1. W przypadku naszych danych, ponieważ składowe RGB pikseli mają wartości z zakresu 0–255, musimy pamiętać o normalizacji danych; można użyć do tego warstwy skalującej wartości.
2. Część wykrywająca elementy obrazu składa się z warstw konwolucyjnych, najczęściej przepla- tanych warstwami zbierającymi:
- głównymi parametrami warstw konwolucyjnych są liczba filtrów i rozmiar filtra; za- zwyczaj zaczynamy od względnie niskiej liczby filtrów (np. 32) o większym rozmiarze (np. 7 × 7), aby wykryć elementarne komponenty obrazu, a na kolejnych warstwach łączymy je w bardziej złożone struktury – kombinacji jest więcej, a więc mamy coraz więcej filtrów, ale mogą być mniejszego rozmiaru (np. 3 × 3),
- zwyczajowo na jedną warstwę konwolucyjną przypadała jedna warstwa zbierająca (zm- niejszająca rozmiar „obrazu”), ale często stosujemy też kilka (np. 2) warstw kon- wolucyjnych bezpośrednio na sobie.
3. Po części konwolucyjnej typowo następuje część gęsta, złożona z warstw gęstych i opcjonalnie regularyzacyjnych (dropout?):
- część gęsta musi być poprzedzona warstwą spłaszczającą dane, gdyż spodziewa się 1- wymiarowej struktury,
- ostatnia warstwa musi być dostosowana do charakterystyki zbioru danych.

In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(scale=1. / 127.5, offset=-1, input_shape=[224, 224, 3]),
  tf.keras.layers.Conv2D(32, 7, activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, 5, activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(96, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation='relu'),
tf.keras.layers.Dropout(0.5),


  tf.keras.layers.Dense(5, activation="softmax")

])


In [16]:
model.compile(loss=["sparse_categorical_crossentropy"], metrics=["accuracy"], optimizer="Adam")
model.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10
230/230 [==============================] - 202s 874ms/step - loss: 1.5558 - accuracy: 0.3100 - val_loss: 1.3335 - val_accuracy: 0.4392
Epoch 2/10
230/230 [==============================] - 206s 894ms/step - loss: 1.3701 - accuracy: 0.4168 - val_loss: 1.2793 - val_accuracy: 0.4664
Epoch 3/10
230/230 [==============================] - 203s 876ms/step - loss: 1.2364 - accuracy: 0.4822 - val_loss: 1.1242 - val_accuracy: 0.5572
Epoch 4/10
230/230 [==============================] - 215s 931ms/step - loss: 1.0636 - accuracy: 0.5661 - val_loss: 1.0229 - val_accuracy: 0.5935
Epoch 5/10
230/230 [==============================] - 201s 870ms/step - loss: 0.9290 - accuracy: 0.6337 - val_loss: 1.0289 - val_accuracy: 0.5717
Epoch 6/10
230/230 [==============================] - 203s 877ms/step - loss: 0.7690 - accuracy: 0.6973 - val_loss: 1.0556 - val_accuracy: 0.5935
Epoch 7/10
230/230 [==============================] - 201s 869ms/step - loss: 0.6202 - accuracy: 0.7624 - val_loss: 1.2419 -

Zapisz wynik ewaluacji dla zbioru uczącego, walidacyjnego i testowego w postaci krotki
(acc_train, acc_valid, acc_test) do pikla simple_cnn_acc.pkl.

In [17]:
import pickle

acc = (model.evaluate(train_set)[1], model.evaluate(valid_set)[1], model.evaluate(test_set)[1])

with open('simple_cnn_acc.pkl', 'wb') as file:
    pickle.dump(acc, file)
acc

31/31 [==============================] - 6s 193ms/step - loss: 1.0391 - accuracy: 0.6104


(0.9026162624359131, 0.5462794899940491, 0.6103542447090149)

## Uczenie transferowe

Tym razem wykorzystamy gotową, dużo bardziej złożoną sieć. Dzięki temu, że sieć będzie zainicjalizowana wagami, możemy znacząco skrócić czas uczenia.
Jako bazową wykorzystamy względnie nowoczesną sieć Xception. Jest ona dostępna w pakiecie
tf.keras.applications.xception.
Wykorzystamy wcześniej już załadowane surowe zbiory danych (..._set_raw).

### Przygotowanie danych

Gotowe modele często dostarczają własnych funkcji przygotowujących wejście w sposób zapewniający optymalne przetwarzanie. Musimy więc zmienić nieco funkcję przygotowującą dane, dodając
wywołanie odpowiedniej metody

In [18]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

Zobaczmy jak tym razem wyglądają wstępnie przetworzone dane; zwróć uwagę, że ponieważ teraz
wartości należą już do zakresu (−1, 1), musimy je odpowiednio przeskalować (ale w sieci nie
będziemy potrzebowali warstwy skalującej):

In [ ]:
plt.figure(figsize=(8, 8))
sample_batch = train_set.take(1)
for X_batch, y_batch in sample_batch:
    for index in range(12):
        plt.subplot(3, 4, index + 1)
        plt.imshow(X_batch[index] / 2 + 0.5)
        plt.title("Class: {}".format(class_names[y_batch[index]]))
        plt.axis("off")
plt.show()

### Budowa sieci

Utwórz model bazowy przy pomocy odpowiedniej metody:

In [19]:
base_model = tf.keras.applications.xception.Xception(
    weights="imagenet",
    include_top=False)


Wyjaśnienie:
- argument weights zapewnia inicjalizację wag sieci wynikami uczenia zbiorem ImageNet,
- argument include_top sprawi, że sieć nie będzie posiadała górnych warstw (które musimy
sami dodać, gdyż są specyficzne dla danego problemu).

Możesz wyświetlić strukturę załadowanej sieci:

In [20]:
for index, layer in enumerate(base_model.layers):
    print(index, layer.name)


0 input_2
1 block1_conv1
2 block1_conv1_bn
3 block1_conv1_act
4 block1_conv2
5 block1_conv2_bn
6 block1_conv2_act
7 block2_sepconv1
8 block2_sepconv1_bn
9 block2_sepconv2_act
10 block2_sepconv2
11 block2_sepconv2_bn
12 conv2d_10
13 block2_pool
14 batch_normalization_4
15 add_12
16 block3_sepconv1_act
17 block3_sepconv1
18 block3_sepconv1_bn
19 block3_sepconv2_act
20 block3_sepconv2
21 block3_sepconv2_bn
22 conv2d_11
23 block3_pool
24 batch_normalization_5
25 add_13
26 block4_sepconv1_act
27 block4_sepconv1
28 block4_sepconv1_bn
29 block4_sepconv2_act
30 block4_sepconv2
31 block4_sepconv2_bn
32 conv2d_12
33 block4_pool
34 batch_normalization_6
35 add_14
36 block5_sepconv1_act
37 block5_sepconv1
38 block5_sepconv1_bn
39 block5_sepconv2_act
40 block5_sepconv2
41 block5_sepconv2_bn
42 block5_sepconv3_act
43 block5_sepconv3
44 block5_sepconv3_bn
45 add_15
46 block6_sepconv1_act
47 block6_sepconv1
48 block6_sepconv1_bn
49 block6_sepconv2_act
50 block6_sepconv2
51 block6_sepconv2_bn
52 block6

Korzystając z API funkcyjnego Keras dodaj warstwy:
- uśredniającą wartości wszystkich „pikseli”,
- wyjściową, gęstą, odpowiednią dla problemu.


In [22]:
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

Przeprowadź uczenie w dwóch krokach:
1. Kilka (np. 5) iteracji, podczas których warstwy sieci bazowej będą zablokowane; ten krok
jest konieczny aby zapobiec „zepsuciu” wag dostarczonych wraz z siecią bazową ze względu
na spodziewane duże błędy wynikające z braku przyuczenia „nowych” warstw:

In [23]:
for layer in base_model.layers:
    layer.trainable = False
model.compile(loss="sparse_categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set,epochs=5)

Epoch 1/5
230/230 [==============================] - 240s 1s/step - loss: 4.2181 - accuracy: 0.3532 - val_loss: 3.4585 - val_accuracy: 0.3521
Epoch 2/5
230/230 [==============================] - 240s 1s/step - loss: 2.9127 - accuracy: 0.4539 - val_loss: 3.2938 - val_accuracy: 0.4446
Epoch 3/5
230/230 [==============================] - 328s 1s/step - loss: 2.5220 - accuracy: 0.5025 - val_loss: 4.9533 - val_accuracy: 0.4229
Epoch 4/5
230/230 [==============================] - 243s 1s/step - loss: 2.2419 - accuracy: 0.5338 - val_loss: 2.5389 - val_accuracy: 0.4973
Epoch 5/5
230/230 [==============================] - 258s 1s/step - loss: 2.0156 - accuracy: 0.5683 - val_loss: 2.8403 - val_accuracy: 0.4628


In [ ]:
for layer in base_model.layers:
    layer.trainable = True
model.fit(train_set, validation_data=valid_set,epochs=10)

Epoch 1/10
230/230 [==============================] - 246s 1s/step - loss: 1.8755 - accuracy: 0.5927 - val_loss: 3.4620 - val_accuracy: 0.4882
Epoch 2/10
230/230 [==============================] - 248s 1s/step - loss: 1.7297 - accuracy: 0.6039 - val_loss: 2.2371 - val_accuracy: 0.4955
Epoch 3/10
230/230 [==============================] - 251s 1s/step - loss: 1.4579 - accuracy: 0.6479 - val_loss: 2.7337 - val_accuracy: 0.4755
Epoch 4/10
212/230 [==========================>...] - ETA: 24s - loss: 1.5082 - accuracy: 0.6360

Zapisz wynik ewaluacji dla zbioru uczącego, walidacyjnego i testowego w postaci krotki
(acc_train, acc_valid, acc_test) do pikla xception_acc.pkl.

In [ ]:
acc = (model.evaluate(train_set)[1], model.evaluate(valid_set)[1], model.evaluate(test_set)[1])

with open('xception_acc.pkl', 'wb') as file:
    pickle.dump(acc, file)
acc